In [5]:
import pandas as pd
from time import sleep
from json import dumps
import datetime as dt
from kafka import KafkaProducer
import pandas as pd
from pytz import timezone
import random

In [6]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer


def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
#         print('Message published successfully. Data: ' )
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def rand_sample(a,b):
    return random.randrange(a,b)      

In [7]:
if __name__ == '__main__': 
    

    topic = 'process'    
    df_process = pd.read_csv('Streaming_Linux_process.csv')
    
    print('Publishing records..')
    producer = connect_kafka_producer()  
    
    Initial = {4:0,5:0,6:0,7:0,8:0}
    Start = {4:0,5:0,6:0,7:0,8:0}    
    end =   {4:0,5:0,6:0,7:0,8:0}
    limit = {4:0,5:0,6:0,7:0,8:0}

    for value in df_process['machine'].unique():
        temp = df_process[df_process['machine']==int(value) ].copy()
        Initial[value]=temp.index[0]
        Start[value]=temp.index[0]
        limit[value] =temp.index[-1]

    while True:
        mesg = pd.DataFrame()

        for value in df_process['machine'].unique():
            
            ran = rand_sample(10,50) 
            end[value] = Start[value]+ran

            temp = df_process[df_process['machine']==int(value)].copy()

            if (int(end[value]) >= int(limit[value])): 
                end[value] = Initial[value] + ran
                Start[value] = Initial[value]

            mesg = mesg.append(temp.loc[Start[value]:end[value],].copy())

            Start[value] = end[value]+1
            
        ts = int(dt.datetime.now(timezone('UTC')).timestamp())
        mesg['ts'] = ts

        message = []

        for col in mesg.columns:
            mesg[col] = mesg[col].astype('str')   

        for ind in mesg.index:
            message.append(dict(mesg.loc[ind,]))
        publish_message(producer_instance=producer, topic_name=topic, data=message)
        sleep(5) 

Publishing records..
Exception in publishing message.
KafkaTimeoutError: Failed to update metadata after 60.0 secs.
Exception in publishing message.
KafkaTimeoutError: Failed to update metadata after 60.0 secs.


KeyboardInterrupt: 